In [2]:
import os
from scipy.io import wavfile
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from cfg import Config
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight as sklearn_compute_class_weight
from tqdm import tqdm
from python_speech_features import mfcc

In [11]:
def build_predictions(audio_dir) :
    y_true = []
    y_pred = []
    fn_prob = {}

    print('Extracting features from audio')
    for fn in tqdm(os.listdir(audio_dir)) :
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        label = fn2class[fn]
        c = classes.index(label)
        y_prob = []

        for i in range(0, wav.shape[0]-config.step, config.step) : 
            sample = wav[i:i+config.step]
            x = mfcc(sample, rate, numcep = config.nfeat, nfilt=config.nfilt, nfft=config.nfft)
            x = (x - config.min) / (config.max - config.min)

            if config.mode == 'conv':
                x = x.reshape(1, x.shape[1], x.shape[0], 1)
            elif config.mode == 'time':
                x = np.expand_dims(x, axis=0)
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
        
        fn_prob[fn] = np.mean(y_prob, axis=0).flatten()

    return y_true, y_pred, fn_prob

  

In [12]:
df = pd.read_csv('instruments.csv')
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname,df.label))
p_path = os.path.join('pickles','conv.p')

with open(p_path, 'rb') as handle :
    config = pickle.load(handle)

model = load_model(config.model_path)

y_true, y_pred, fn_prob = build_predictions('clean')
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)

y_probs = []
for i, row in df.iterrows():
    y_prob = fn_prob[row.fname]
    y_probs.append()
    for c, p in zip(classes, y_prob):
        df.at[i,c] = p

y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred
df.to_csv('prediction.csv', index=False)

Extracting features from audio


  0%|          | 0/300 [00:00<?, ?it/s]

1/1 [==============================] - 0s 30ms/step


  0%|          | 1/300 [00:04<24:40,  4.95s/it]

1/1 [==============================] - 0s 33ms/step


...

1/1 [==============================] - 0s 50ms/step


100%|█████████▉| 299/300 [27:29<00:05,  5.45s/it]

1/1 [==============================] - 0s 55ms/step


100%|██████████| 300/300 [27:39<00:00,  5.53s/it]


TypeError: list.append() takes exactly one argument (0 given)